#  Todo
* ~How do we keep an option to hover on the countries that have no data?~
* ~How do we hide the countries from the map that are *never* a part of the datasets? (Russia, Iceland etc)~
* How do we make a slider work?
* Should we do a "dias show"? In that case: How?
* Which datasets should we work with and which points do we want to make?

In [210]:
import pandas as pd

df = pd.read_csv("drug_offences.csv", sep=";")
df.head(3)

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,Unnamed: 13
0,Austria,47880.0,42610.0,36235.0,32907.0,30250.0,28227.0,23797.0,25892.0,23853.0,22729.0,20043.0,24166.0,NaN
1,Belgium,54749.0,51774.0,49416.0,47083.0,48727.0,42935.0,39181.0,41661.0,39307.0,41525.0,40357.0,37867.0,NaN
2,Bulgaria,2382.0,2433.0,4886.0,4195.0,9340.0,9521.0,8322.0,6167.0,6577.0,5149.0,4185.0,4301.0,NaN


In [211]:
empty_df = df.loc[df['2018'].isnull()].copy()
empty_df["2018"] = 0
empty_df = empty_df.append({"2018": 0, "Country":"Switzerland"}, ignore_index=True)
empty_df

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,Unnamed: 13
0,Czechia,0,NaN,5564.0,5549.0,7438.0,6803.0,5317.0,5003.0,4200.0,3821.0,3491.0,NaN,NaN
1,Greece,0,17995.0,17741.0,23748.0,22422.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Slovakia,0,1692.0,NaN,969.0,1147.0,1191.0,1214.0,1204.0,1135.0,1079.0,912.0,786.0,NaN
3,Slovenia,0,NaN,4235.0,NaN,4519.0,5329.0,5392.0,5616.0,4174.0,4244.0,5100.0,3720.0,NaN
4,United Kingdom,0,108033.0,106862.0,115377.0,128260.0,139803.0,144434.0,154212.0,152451.0,147013.0,149203.0,135655.0,NaN
5,Switzerland,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
import plotly.express as px
import plotly.graph_objects as go

#df = px.data.gapminder().query("year==2007")


fig = px.choropleth(df, locations="Country",
                    color="2018", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma,
                    scope="europe",
                    locationmode="country names",
                    custom_data=["Country", "2018"])

fig.update_traces(hovertemplate="<b>Country: %{customdata[0]}<br></b><br>" +
                                     "Drug Offenders: %{customdata[1]:n}<br>", selector=dict(type='choropleth'))

fig.add_trace(
go.Choropleth(locations=empty_df["Country"],
                    z=empty_df["2018"], # lifeExp is a column of gapminder
                    colorscale="Greys",
                    text=df['Country'], # hover text
                    locationmode="country names",
                    hovertemplate="<b>Country: %{customdata}<br></b>No data available",
                    customdata=[*empty_df["Country"]],
                    showlegend=None, showscale=False)
)

fig.update_geos(fitbounds="locations", visible=False)

fig2 = px.choropleth(empty_df, locations="Country",
                    color="2018", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma,
                    scope="europe",
                    locationmode="country names",
                    custom_data=["Country", "2018"])

fig.show()

In [213]:
fig3 = go.Figure(go.Choropleth(locations=df["Country"],
                    z=df["2018"], # lifeExp is a column of gapminder
                    colorscale="Greys",
                    locationmode="country names",
                    hovertemplate="<b>Country: %{customdata}<br></b>No data available",
                    customdata=[*df["Country"]],
                    showlegend=None, showscale=False))

fig3.update_geos(fitbounds="locations", visible=False)
fig3.show()

In [216]:
def create_choropleth(df, year, colors="Plasma"):
    return go.Choropleth(locations=df["Country"],
                        z=df[str(year)], # lifeExp is a column of gapminder
                        colorscale=colors,
                        locationmode="country names",
                        customdata=[*zip(df["Country"], df[str(year)])])

def create_missing_choropleth(df, year, colors="Greys"):
    empty_df = df.loc[df[str(year)].isnull()].copy()
    empty_df = empty_df.append({str(year): 0, "Country":"Switzerland"}, ignore_index=True)
    empty_df[str(year)] = 0
    #print(empty_df)
    return go.Choropleth(locations=empty_df["Country"],
                        z=empty_df[str(year)], # lifeExp is a column of gapminder
                        colorscale=colors,
                        locationmode="country names",
                        hovertemplate="<b>Country: %{customdata}<br></b>No data available",
                        customdata=[*empty_df["Country"]],
                        showlegend=None, showscale=False)

year = 2015

fig3 = go.Figure(create_choropleth(df, year))
fig3.update_traces(hovertemplate="<b>Country: %{customdata[0]}<br></b><br>" +
                                     "Drug Offenders: %{customdata[1]:n}<br>", selector=dict(type='choropleth'))
fig3.update_geos(scope="europe")

# Add missing
fig3.add_trace(create_missing_choropleth(df, year, "Greys"))
fig3.update_geos(fitbounds="locations", visible=False)
fig3.show()